In [1]:
def calculate_bloodtype_score(donor_bloodtype, recipient_bloodtype):
    if donor_bloodtype == recipient_bloodtype:
        return 10
    elif donor_bloodtype == 'O':
        return 5
    elif donor_bloodtype == 'A' and recipient_bloodtype in ['A', 'AB']:
        return 5
    elif donor_bloodtype == 'B' and recipient_bloodtype in ['B', 'AB']:
        return 5
    elif donor_bloodtype == 'AB':
        return 0
    elif donor_bloodtype == 'A' and recipient_bloodtype == 'O':
        return 5
    else:
        return 0


Raw Data

In [2]:
import numpy as np
import pandas as pd

# Read Data 
df = pd.read_excel('C:\\Users\\Hongan Li\\Desktop\\Research Project\\liver transplant\\hhhhh.xlsx')
recipients = df.iloc[:4000, :8].reset_index(drop=True)
donors = df.iloc[:4000, 8:].reset_index(drop=True)

# Generate all possible combinations
donor_indices = np.repeat(donors.index.values, recipients.shape[0])
recipient_indices = np.tile(recipients.index.values, donors.shape[0])

In [11]:
df

,Waittime,MELD,MELDNA,TransplantBMI,TransplantAge,Reci_Gender,Reci_Bloodtype,Reci_loca,Donor_BMI,Donor_Age,Donor_Gender,Donor_Bloodtype
0,1,22.939930,22.931727,24.712049,54.291739,Male,O,inpt,21.369229,45.379636,Male,B
1,157,18.985337,21.381548,28.887631,59.817111,Male,A,home,25.408853,60.516200,Male,O
2,49,23.694392,26.243047,26.869533,63.198440,Female,A,home,24.902931,30.989861,Male,A
3,81,11.375444,8.545730,27.411596,28.845900,Male,O,inpt,35.396950,41.142174,Female,B
4,620,26.835544,29.211865,24.530967,55.523918,Male,O,home,29.372668,49.138937,Female,O
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,638,17.050266,14.632579,29.624822,66.768544,Male,A,home,30.205990,62.181956,Female,A
3996,146,13.369472,10.730959,24.984602,61.143847,Male,A,home,18.078771,57.131994,Male,A
3997,60,19.096785,22.031645,31.146902,41.818547,Male,A,inpt,24.720377,39.402747,Female,A
3998,152,16.906836,14.620845,36.196905,36.732881,Male,A,home,22.702255,18.422351,Male,A


Preferences of donors to recicpients

In [5]:
# Create a DataFrame using vectorization operations
all_combinations_4000 = pd.DataFrame({
    'Donor_Index': donor_indices,
    'Recipient_Index': recipient_indices,
})

# Add donors' features
for col in donors.columns:
    all_combinations_4000[col] = donors[col].values[donor_indices]

# Add recipients' features 
for col in recipients.columns:
    all_combinations_4000[col] = recipients[col].values[recipient_indices]


# Generate blood type scores
all_combinations_4000['Bloodtype_score'] = all_combinations_4000.apply(
    lambda row: calculate_bloodtype_score(row['Donor_Bloodtype'], row['Reci_Bloodtype']),
    axis=1
)
all_combinations_4000

# Save the DataFrame as a CSV file
output_path = 'C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\GS+Gender\\all_combinations_4000.csv'
all_combinations_4000.to_csv(output_path, index=True)

In [6]:
from sklearn.preprocessing import StandardScaler

# Create a new DataFrame to save the normalized data.
standardized_combinations_4000 = all_combinations_4000.copy()

# Standardize specific columns
scaler = StandardScaler()
standardized_combinations_4000[['MELD', 'TransplantAge', 'Waittime']] = scaler.fit_transform(
    all_combinations_4000[['MELD', 'TransplantAge', 'Waittime']]
)

# Calculate the initial preference score
standardized_combinations_4000['Initial_Preference'] = (
    0.5 * standardized_combinations_4000['MELD'] +
    0.3 * standardized_combinations_4000['Bloodtype_score'] -
    0.15 * standardized_combinations_4000['TransplantAge'] +
    0.05 * standardized_combinations_4000['Waittime']
)

# Save the DataFrame as a CSV file
output_path = 'C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\GS+Gender\\standardized_combinations_4000.csv'
standardized_combinations_4000.to_csv(output_path, index=True)

In [7]:
standardized_combinations_4000

,Donor_Index,Recipient_Index,Donor_BMI,Donor_Age,Donor_Gender,Donor_Bloodtype,Waittime,MELD,MELDNA,TransplantBMI,TransplantAge,Reci_Gender,Reci_Bloodtype,Reci_loca,Bloodtype_score,Initial_Preference
0,0,0,21.369229,45.379636,Male,B,-0.773793,0.444786,22.931727,24.712049,0.224871,Male,O,inpt,0,0.149973
1,0,1,21.369229,45.379636,Male,B,-0.004461,-0.029561,21.381548,28.887631,0.736610,Male,A,home,0,-0.125495
2,0,2,21.369229,45.379636,Male,B,-0.537075,0.535283,26.243047,26.869533,1.049776,Female,A,home,0,0.083321
3,0,3,21.369229,45.379636,Male,B,-0.379264,-0.942355,8.545730,27.411596,-2.131824,Male,O,inpt,0,-0.170367
4,0,4,21.369229,45.379636,Male,B,2.278879,0.912059,29.211865,24.530967,0.338991,Male,O,home,0,0.519125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15999995,3999,3995,18.639493,39.731841,Female,A,2.367648,-0.261669,14.632579,29.624822,1.380425,Male,A,home,10,2.780484
15999996,3999,3996,18.639493,39.731841,Female,A,-0.058708,-0.703175,10.730959,24.984602,0.859487,Male,A,home,10,2.516554
15999997,3999,3997,18.639493,39.731841,Female,A,-0.482828,-0.016193,22.031645,31.146902,-0.930348,Male,A,inpt,10,3.107314
15999998,3999,3998,18.639493,39.731841,Female,A,-0.029119,-0.278873,14.620845,36.196905,-1.401362,Male,A,home,10,3.069312


In [10]:
import numpy as np

# Select the recipients from all_combinations_4000 where TransplantAge is less than 18
under_18_recipients = all_combinations_4000[all_combinations_4000['TransplantAge'] < 18]

# Get the maximum wait time under 18 years old
max_waittime = under_18_recipients['Waittime'].max()

# Rule 1: The recipients who have waited the longest will receive an additional 10 points
recipient_index_with_max_waittime = under_18_recipients[
    under_18_recipients['Waittime'] == max_waittime
]['Recipient_Index'].unique()

# Give a 10-point bonus to recipients who wait within the maximum waiting time (vectorized operation)
standardized_combinations_4000.loc[
    standardized_combinations_4000['Recipient_Index'].isin(recipient_index_with_max_waittime), 
    'Initial_Preference'
] += 10

# Rule 2: Add points to recipients whose waiting time is less than the maximum waiting time, 
#         uniformly distributed from 1 to 10 points
waittime_range = np.linspace(10, 1, len(under_18_recipients['Waittime'].unique()))
waittime_scores = dict(zip(sorted(under_18_recipients['Waittime'].unique()), waittime_range))

# map is used directly to add points to eligible rows
standardized_combinations_4000['Waittime_bonus'] = standardized_combinations_4000['Waittime'].map(waittime_scores)
standardized_combinations_4000['Waittime_bonus'].fillna(0, inplace=True)  # 没有匹配的填充为 0

# Bonus points for vectorizing blood types
donor_bloodtype = standardized_combinations_4000['Donor_Bloodtype'].values
recipient_bloodtype = standardized_combinations_4000['Reci_Bloodtype'].values

# Rules 3, 4, 5, 6: Vectorization calculations based on blood type
bloodtype_bonus = np.zeros_like(donor_bloodtype, dtype=float)

# Calculate the same blood type plus 10 points
same_bloodtype = donor_bloodtype == recipient_bloodtype
bloodtype_bonus[same_bloodtype] = 10

# Blood group compatibility (donor O or donor A and recipient O) plus 5 points
compatible_bloodtype = (donor_bloodtype == 'O') | ((donor_bloodtype == 'A') & (recipient_bloodtype == 'O'))
bloodtype_bonus[compatible_bloodtype] = 5

# Blood type bonus points were added to standardized_combinations_4000
standardized_combinations_4000['Bloodtype_bonus'] = bloodtype_bonus

# Finally, update the Initial_Preference column
standardized_combinations_4000['Initial_Preference'] += standardized_combinations_4000['Waittime_bonus'] + standardized_combinations_4000['Bloodtype_bonus']

# Delete temporary column
standardized_combinations_4000.drop(columns=['Waittime_bonus', 'Bloodtype_bonus'], inplace=True)

# Save the DataFrame as a CSV file
output_path = 'C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\GS+Gender\\standardized_combinations_4000.csv'
standardized_combinations_4000.to_csv(output_path, index=True)

In [12]:
standardized_combinations_4000

,Donor_Index,Recipient_Index,Donor_BMI,Donor_Age,Donor_Gender,Donor_Bloodtype,Waittime,MELD,MELDNA,TransplantBMI,TransplantAge,Reci_Gender,Reci_Bloodtype,Reci_loca,Bloodtype_score,Initial_Preference
0,0,0,21.369229,45.379636,Male,B,-0.773793,0.444786,22.931727,24.712049,0.224871,Male,O,inpt,0,0.149973
1,0,1,21.369229,45.379636,Male,B,-0.004461,-0.029561,21.381548,28.887631,0.736610,Male,A,home,0,-0.125495
2,0,2,21.369229,45.379636,Male,B,-0.537075,0.535283,26.243047,26.869533,1.049776,Female,A,home,0,0.083321
3,0,3,21.369229,45.379636,Male,B,-0.379264,-0.942355,8.545730,27.411596,-2.131824,Male,O,inpt,0,-0.170367
4,0,4,21.369229,45.379636,Male,B,2.278879,0.912059,29.211865,24.530967,0.338991,Male,O,home,0,0.519125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15999995,3999,3995,18.639493,39.731841,Female,A,2.367648,-0.261669,14.632579,29.624822,1.380425,Male,A,home,10,12.780484
15999996,3999,3996,18.639493,39.731841,Female,A,-0.058708,-0.703175,10.730959,24.984602,0.859487,Male,A,home,10,12.516554
15999997,3999,3997,18.639493,39.731841,Female,A,-0.482828,-0.016193,22.031645,31.146902,-0.930348,Male,A,inpt,10,13.107314
15999998,3999,3998,18.639493,39.731841,Female,A,-0.029119,-0.278873,14.620845,36.196905,-1.401362,Male,A,home,10,13.069312


In [13]:
#donors' Preference_Score order
recipients_names = [f'r{i+1}' for i in range(4000)]
donors_names = [f'd{i+1}' for i in range(4000)]
don_pre_arry = standardized_combinations_4000['Initial_Preference'].values.reshape(4000, 4000)
#don_pre_pd = pd.DataFrame(don_pre_arry, index=recipients_names, columns=donors_names)
sorted_indices = np.argsort(don_pre_arry, axis=1)
ranks = np.argsort(sorted_indices, axis=1) + 1
don_ranks = ranks.max(axis=1)[:, None] + 1 - ranks
don_pre_new_4000 = pd.DataFrame(don_ranks, index=donors_names, columns=recipients_names)

# Save the DataFrame as a CSV file
output_path = 'C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\GS+Gender\\don_pre_new_4000.csv'
don_pre_new_4000.to_csv(output_path, index=True)  # 这里将index设置为True以保留行名

don_pre_new_4000

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,...,r3991,r3992,r3993,r3994,r3995,r3996,r3997,r3998,r3999,r4000
d1,1841,2593,2008,2727,1216,3968,162,3387,1894,3298,...,2635,3279,2384,3339,2891,2848,3399,1950,2047,1965
d2,589,2988,2592,1067,283,1948,2295,3520,2507,3463,...,3020,1390,872,1426,1162,3145,3536,2552,2621,2562
d3,2178,911,618,2625,1881,3259,3617,1298,557,1255,...,935,2904,2444,2933,2712,1032,1304,592,642,600
d4,1841,2593,2008,2727,1216,3968,162,3387,1894,3298,...,2635,3279,2384,3339,2891,2848,3399,1950,2047,1965
d5,589,2988,2592,1067,283,1948,2295,3520,2507,3463,...,3020,1390,872,1426,1162,3145,3536,2552,2621,2562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d3996,2178,911,618,2625,1881,3259,3617,1298,557,1255,...,935,2904,2444,2933,2712,1032,1304,592,642,600
d3997,2178,911,618,2625,1881,3259,3617,1298,557,1255,...,935,2904,2444,2933,2712,1032,1304,592,642,600
d3998,2178,911,618,2625,1881,3259,3617,1298,557,1255,...,935,2904,2444,2933,2712,1032,1304,592,642,600
d3999,2178,911,618,2625,1881,3259,3617,1298,557,1255,...,935,2904,2444,2933,2712,1032,1304,592,642,600


In [15]:
don_pre_new_4000.iloc[:20, :20]

,r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,r14,r15,r16,r17,r18,r19,r20
d1,1841,2593,2008,2727,1216,3968,162,3387,1894,3298,2726,1712,2684,2864,999,559,2625,2923,2323,129
d2,589,2988,2592,1067,283,1948,2295,3520,2507,3463,3069,2381,3046,1148,162,1811,1005,1178,2819,2141
d3,2178,911,618,2625,1881,3259,3617,1298,557,1255,974,461,955,2700,1762,3286,2567,2723,783,3584
d4,1841,2593,2008,2727,1216,3968,162,3387,1894,3298,2726,1712,2684,2864,999,559,2625,2923,2323,129
d5,589,2988,2592,1067,283,1948,2295,3520,2507,3463,3069,2381,3046,1148,162,1811,1005,1178,2819,2141
d6,589,2988,2592,1067,283,1948,2295,3520,2507,3463,3069,2381,3046,1148,162,1811,1005,1178,2819,2141
d7,1841,2593,2008,2727,1216,3968,162,3387,1894,3298,2726,1712,2684,2864,999,559,2625,2923,2323,129
d8,2178,911,618,2625,1881,3259,3617,1298,557,1255,974,461,955,2700,1762,3286,2567,2723,783,3584
d9,589,2988,2592,1067,283,1948,2295,3520,2507,3463,3069,2381,3046,1148,162,1811,1005,1178,2819,2141
d10,2178,911,618,2625,1881,3259,3617,1298,557,1255,974,461,955,2700,1762,3286,2567,2723,783,3584


Preferences of recipients to donors

In [3]:
import pandas as pd
import numpy as np

# Read Data 
recipients = df.iloc[:4000, :8].reset_index(drop=True)
donors = df.iloc[:4000, 8:].reset_index(drop=True)

# Generate all possible combinations
recipient_indices = np.repeat(recipients.index.values, donors.shape[0])
donor_indices = np.tile(donors.index.values, recipients.shape[0])

# Create a DataFrame using vectorization operations
all_combinations1_4000 = pd.DataFrame({
    'Recipient_Index': recipient_indices,
    'Donor_Index': donor_indices,
})

# Use the broadcast directly to add additional characteristics of the recipient
for col in recipients.columns:
    all_combinations1_4000[col] = recipients[col].values[recipient_indices]

# Add donor features
for col in donors.columns:
    all_combinations1_4000[col] = donors[col].values[donor_indices]

all_combinations1_4000

# Save the DataFrame as a CSV file
output_path = 'C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\GS+Gender\\all_combinations1_4000.csv'
all_combinations1_4000.to_csv(output_path, index=True)


In [17]:
import pandas as pd
import numpy as np

# Define the baseline survival rate for the survival rate difference
base_survival_rate = 72

# Calculate the survival rate difference for the feature of recipients
def calculate_recipient_diff(df):
    recipient_diff = np.zeros(len(df), dtype=np.float32)

    # 1. TransplantAge is less than or equal to 65 and greater than 65
    recipient_diff += np.where(df['TransplantAge'] <= 65, 1, -15)

    # 2. Reci_Gender：Female 和 Male
    recipient_diff += np.where(df['Reci_Gender'] == 'Female', 4, -3)

    # 3. MELD score
    recipient_diff += np.where(df['MELD'] <= 6, 8, 0)
    recipient_diff += np.where((df['MELD'] > 6) & (df['MELD'] <= 15), 3, 0)
    recipient_diff += np.where((df['MELD'] > 15) & (df['MELD'] <= 25), 1, 0)
    recipient_diff += np.where(df['MELD'] > 25, -10, 0)

    # 4. Reci_loca
    recipient_diff += np.where(df['Reci_loca'] == 'inpt', -3, 0)
    recipient_diff += np.where(df['Reci_loca'] == 'home', 4, 0)
    recipient_diff += np.where(df['Reci_loca'] == 'icu', -8, 0)

    return recipient_diff

# Calculating the survival rate difference for donor characteristics
def calculate_donor_diff(df):
    donor_diff = np.zeros(len(df), dtype=np.float32)

    # 1. Donor_Age
    donor_diff += np.where(df['Donor_Age'] <= 50, 3, -6)

    return donor_diff


In [18]:
import pandas as pd
import numpy as np

# Define the baseline survival rate for the survival rate difference
base_survival_rate = 72

# Calculating the survival rate difference between recipients and donors' characteristics
def calculate_diff(df):
    diff = np.zeros(len(df), dtype=np.float32)

    # 1. Reci_Gender matches Donor_Gender
    female_female = (df['Reci_Gender'] == 'Female') & (df['Donor_Gender'] == 'Female')
    female_male = (df['Reci_Gender'] == 'Female') & (df['Donor_Gender'] == 'Male')
    male_female = (df['Reci_Gender'] == 'Male') & (df['Donor_Gender'] == 'Female')
    male_male = (df['Reci_Gender'] == 'Male') & (df['Donor_Gender'] == 'Male')
    
    diff += np.where(female_female, 6, 0)
    diff += np.where(female_male, -6, 0)
    diff += np.where(male_female, 6, 0)
    diff += np.where(male_male, 1, 0)

    # 2. Donor_Age matches
    diff += np.where(df['Donor_Age'] < 60, 3, 0)
    diff += np.where(df['Donor_Age'] >= 60, 10, 0)

    return diff

# Define the blood type matching bonus rules
def calculate_bloodtype_bonus(recipient_bloodtype, donor_bloodtype):
    if recipient_bloodtype == donor_bloodtype:
        return 10  # Same blood type, plus 10 points
    elif recipient_bloodtype == 'O':
        return 5  # 5 points for recipient O, donor A, B, AB
    elif recipient_bloodtype == 'A' and donor_bloodtype == 'AB':
        return 5  # 5 points for recipient A, donor AB
    elif recipient_bloodtype == 'B' and donor_bloodtype == 'AB':
        return 5  # 5 points for recipient B, donor AB
    elif recipient_bloodtype == 'O' and donor_bloodtype == 'A':
        return 5  # 5 points for a recipient of O and a donor of A
    else:
        return 0  # Add 0 points for incompatible blood types

In [4]:
# 基础生存率
base_survival_rate = 72

# 定义计算血型匹配分的函数
def bloodtype_score(recipient, donor):
    if recipient == donor:
        return 10
    elif recipient == 'O':
        return 5 if donor in ['A', 'B', 'AB'] else 0
    elif recipient == 'A' and donor in ['AB']:
        return 5
    elif recipient == 'B' and donor in ['AB']:
        return 5
    elif (recipient == 'A' and donor == 'B') or (recipient == 'B' and donor == 'A'):
        return 0
    return 0

# 定义计算生存率差值的函数
def survival_rate_diff(row):
    diff = 0
    # 性别和年龄的条件
    if row['Reci_Gender'] == 'Female' and row['Donor_Gender'] == 'Female':
        diff += 2
    elif row['Reci_Gender'] == 'Female' and row['Donor_Gender'] == 'Male':
        diff -= 3
    elif row['Reci_Gender'] == 'Male' and row['Donor_Gender'] == 'Female':
        diff += 0
    elif row['Reci_Gender'] == 'Male' and row['Donor_Gender'] == 'Male':
        diff += 1

    # 捐赠者年龄的条件
    if row['Donor_Age'] <= 50:
        diff += 3
    else:
        diff -= 6

    # 接受者年龄的条件
    if row['TransplantAge'] <= 65:
        diff += 1
    else:
        diff -= 15

    # MELD分数的条件
    if row['MELD'] >= 25:
        diff -= 10
    elif row['MELD'] >= 16:
        diff += 1
    elif row['MELD'] >= 6:
        diff += 3

    # 接受者位置的条件
    if row['Reci_loca'] == 'icu':
        diff -= 3
    elif row['Reci_loca'] == 'home':
        diff += 2
    elif row['Reci_loca'] == 'inpt':
        diff -= 5

    # 添加血型匹配分
    diff += bloodtype_score(row['Reci_Bloodtype'], row['Donor_Bloodtype'])

    # 返回总生存率
    return base_survival_rate + diff

# 应用上述函数计算每个组合的偏好分数
all_combinations1_4000['Preference_Score'] = all_combinations1_4000.apply(survival_rate_diff, axis=1)


In [5]:

# 保存 DataFrame 为 CSV 文件
output_path = 'C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\GS+Gender\\all_combinations1_4000.csv'
all_combinations1_4000.to_csv(output_path, index=True)

In [19]:
# 优化数据类型
all_combinations1_4000['TransplantAge'] = all_combinations1_4000['TransplantAge'].astype('float32')
all_combinations1_4000['MELD'] = all_combinations1_4000['MELD'].astype('float32')
all_combinations1_4000['Donor_Age'] = all_combinations1_4000['Donor_Age'].astype('float32')

# 将数据分成多个批次处理
batch_size = 1000000  # 每次处理100万条记录
num_batches = len(all_combinations1_4000) // batch_size + 1

# 初始化偏好分数列
all_combinations1_4000['Preference_Score'] = 0

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(all_combinations1_4000))
    
    # 处理每个批次的生存率差值
    batch_df = all_combinations1_4000.iloc[start_idx:end_idx]
    diff = calculate_diff(batch_df)

    # 计算初始偏好分数
    all_combinations1_4000.loc[start_idx:end_idx - 1, 'Preference_Score'] = (
        base_survival_rate + diff
    )

    # 根据血型加分
    bloodtype_bonus = batch_df.apply(lambda row: calculate_bloodtype_bonus(row['Reci_Bloodtype'], row['Donor_Bloodtype']), axis=1)
    
    # 将血型加分应用到偏好分数，并确保类型为 float32
    all_combinations1_4000.loc[start_idx:end_idx - 1, 'Preference_Score'] += bloodtype_bonus.astype('float32')

In [6]:
all_combinations1_4000

,Recipient_Index,Donor_Index,Waittime,MELD,MELDNA,TransplantBMI,TransplantAge,Reci_Gender,Reci_Bloodtype,Reci_loca,Donor_BMI,Donor_Age,Donor_Gender,Donor_Bloodtype,Preference_Score
0,0,0,1,22.939930,22.931727,24.712049,54.291739,Male,O,inpt,21.369229,45.379636,Male,B,78
1,0,1,1,22.939930,22.931727,24.712049,54.291739,Male,O,inpt,25.408853,60.516200,Male,O,74
2,0,2,1,22.939930,22.931727,24.712049,54.291739,Male,O,inpt,24.902931,30.989861,Male,A,78
3,0,3,1,22.939930,22.931727,24.712049,54.291739,Male,O,inpt,35.396950,41.142174,Female,B,77
4,0,4,1,22.939930,22.931727,24.712049,54.291739,Male,O,inpt,29.372668,49.138937,Female,O,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15999995,3999,3995,138,14.987741,12.505690,23.126541,25.789776,Male,A,home,30.205990,62.181956,Female,A,82
15999996,3999,3996,138,14.987741,12.505690,23.126541,25.789776,Male,A,home,18.078771,57.131994,Male,A,83
15999997,3999,3997,138,14.987741,12.505690,23.126541,25.789776,Male,A,home,24.720377,39.402747,Female,A,91
15999998,3999,3998,138,14.987741,12.505690,23.126541,25.789776,Male,A,home,22.702255,18.422351,Male,A,92


In [7]:
#recipients' Preference_Score order
recipients_names = [f'r{i+1}' for i in range(4000)]
donors_names = [f'd{i+1}' for i in range(4000)]
reci_pre_arry = all_combinations1_4000['Preference_Score'].values.reshape(4000, 4000)
reci_pre_pd = pd.DataFrame(reci_pre_arry, index=recipients_names, columns=donors_names)
sorted_indices1 = np.argsort(reci_pre_arry, axis=1)
ranks1 = np.argsort(sorted_indices1, axis=1) + 1
reci_ranks1 = ranks1.max(axis=1)[:, None] + 1 - ranks1
reci_pre_new_4000 = pd.DataFrame(reci_ranks1, index=recipients_names, columns=donors_names)

# 保存 DataFrame 为 CSV 文件
output_path = 'C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\GS+Gender\\reci_pre_new_4000.csv'
reci_pre_new_4000.to_csv(output_path, index=True)  # 这里将index设置为True以保留行名

reci_pre_new_4000

,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,...,d3991,d3992,d3993,d3994,d3995,d3996,d3997,d3998,d3999,d4000
r1,1433,2868,1611,1954,805,588,3239,1955,796,1612,...,3715,2984,2981,799,582,3722,3246,2429,1425,2156
r2,2052,3117,400,2841,2844,1793,3125,884,2845,402,...,1800,3639,3626,2847,1795,1794,1323,878,61,709
r3,2314,3495,834,1485,1478,3066,3519,412,1520,827,...,1026,3107,3119,1480,3065,1038,2249,427,865,1
r4,1433,2868,1611,1954,805,588,3239,1955,796,1612,...,3715,2984,2981,799,582,3722,3246,2429,1425,2156
r5,1433,2868,1611,1954,805,588,3239,1955,796,1612,...,3715,2984,2981,799,582,3722,3246,2429,1425,2156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
r3996,2052,3117,400,2841,2844,1793,3125,884,2845,402,...,1800,3639,3626,2847,1795,1794,1323,878,61,709
r3997,2052,3117,400,2841,2844,1793,3125,884,2845,402,...,1800,3639,3626,2847,1795,1794,1323,878,61,709
r3998,2052,3117,400,2841,2844,1793,3125,884,2845,402,...,1800,3639,3626,2847,1795,1794,1323,878,61,709
r3999,2052,3117,400,2841,2844,1793,3125,884,2845,402,...,1800,3639,3626,2847,1795,1794,1323,878,61,709


Random Effect Linear Regression Model

In [31]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Vectorized computation of Initial_Diff
def compute_initial_diff_vectorized(df):
    # Start with a diff column initialized to zero
    diff = np.zeros(len(df))
    
    # Gender condition
    diff += np.where((df['Reci_Gender'] == 'Female') & (df['Donor_Gender'] == 'Female'), 2, 0)
    diff += np.where((df['Reci_Gender'] == 'Female') & (df['Donor_Gender'] == 'Male'), -3, 0)
    diff += np.where((df['Reci_Gender'] == 'Male') & (df['Donor_Gender'] == 'Female'), 0, 0)
    diff += np.where((df['Reci_Gender'] == 'Male') & (df['Donor_Gender'] == 'Male'), 1, 0)

    # Donor age condition
    diff += np.where(df['Donor_Age'] <= 50, 3, -6)

    # Recipient age condition
    diff += np.where(df['TransplantAge'] <= 65, 1, -15)

    # MELD score condition
    diff += np.where(df['MELD'] >= 25, -10, 0)
    diff += np.where((df['MELD'] >= 16) & (df['MELD'] < 25), 1, 0)
    diff += np.where((df['MELD'] >= 6) & (df['MELD'] < 16), 3, 0)

    # Reci_loca condition
    diff += np.where(df['Reci_loca'] == 'icu', -13, 0)
    diff += np.where(df['Reci_loca'] == 'home', 2, 0)
    diff += np.where(df['Reci_loca'] == 'inpt', -5, 0)

    return diff

# Divide and conquer large data sets
chunksize = 10000  # Process 10,000 rows each time
train_chunks = []
test_chunks = []

# Read the file and perform block processing
for chunk in pd.read_csv('all_combinations1_4000.csv', chunksize=chunksize):
    # Calculate Initial_Diff
    chunk['Initial_Diff'] = compute_initial_diff_vectorized(chunk)
    
    # Split the data with ‘Reci_loca’ of ‘ventilated’ into training and test sets
    train_chunk = chunk[chunk['Reci_loca'] != 'ventilated']
    test_chunk = chunk[chunk['Reci_loca'] == 'ventilated']
    
    # Save the block result
    train_chunks.append(train_chunk)
    test_chunks.append(test_chunk)

# Combine the data after it has been split into blocks
train_data = pd.concat(train_chunks, ignore_index=True)
test_data = pd.concat(test_chunks, ignore_index=True)

# Extract features and target variables
X_train = train_data[['Reci_Gender', 'Donor_Gender', 'Donor_Age', 'TransplantAge', 'MELD', 'Reci_loca']]
y_train = train_data['Initial_Diff']

# Preprocess and Regression Pipeline
numerical_features = ['Donor_Age', 'TransplantAge', 'MELD']
categorical_features = ['Reci_Gender', 'Donor_Gender', 'Reci_loca']

# Create ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Create Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', SGDRegressor(max_iter=1000, tol=1e-3))
])

# Train the regression model
pipeline.fit(X_train, y_train)

# Predict the difference between the ‘ventilated’ samples
X_test = test_data[['Reci_Gender', 'Donor_Gender', 'Donor_Age', 'TransplantAge', 'MELD', 'Reci_loca']]
predicted_diff = pipeline.predict(X_test)

# Assign the predicted difference to test_data
test_data['Predicted_Diff'] = predicted_diff

# Initial_Diff of train_data as Predicted_Diff
train_data['Predicted_Diff'] = train_data['Initial_Diff']

# Combine train_data and test_data
all_combinations1_4000 = pd.concat([train_data, test_data], ignore_index=True)

# Calculate blood type matching scores
def bloodtype_score(recipient, donor):
    if recipient == donor:
        return 10
    elif recipient == 'O' and donor in ['A', 'B', 'AB']:
        return 5
    elif recipient == 'A' and donor == 'AB':
        return 5
    elif recipient == 'B' and donor == 'AB':
        return 5
    else:
        return 0

# Calculate Bloodtype_Score
all_combinations1_4000['Bloodtype_Score'] = all_combinations1_4000.apply(
    lambda row: bloodtype_score(row['Reci_Bloodtype'], row['Donor_Bloodtype']), axis=1)

# Calculate the final Preference_Score
base_survival_rate = 72
all_combinations1_4000['Preference_Score'] = base_survival_rate +  + all_combinations1_4000['Bloodtype_Score']


In [32]:
all_combinations1_4000['Predicted_Diff']

0            1.000000
1           -8.000000
2            1.000000
3            0.000000
4            0.000000
              ...    
15999995   -21.879198
15999996   -21.621564
15999997   -17.455910
15999998   -14.104889
15999999   -17.519814
Name: Predicted_Diff, Length: 16000000, dtype: float64

In [33]:
#recipients' Preference_Score order
recipients_names = [f'r{i+1}' for i in range(4000)]
donors_names = [f'd{i+1}' for i in range(4000)]
reci_pre_arry = all_combinations1_4000['Preference_Score'].values.reshape(4000, 4000)
reci_pre_pd = pd.DataFrame(reci_pre_arry, index=recipients_names, columns=donors_names)
sorted_indices1 = np.argsort(reci_pre_arry, axis=1)
ranks1 = np.argsort(sorted_indices1, axis=1) + 1
reci_ranks1 = ranks1.max(axis=1)[:, None] + 1 - ranks1
reci_pre_new_4000_4 = pd.DataFrame(reci_ranks1, index=recipients_names, columns=donors_names)

# Save the DataFrame as a CSV file
output_path = 'C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\GS+Gender\\reci_pre_new_4000_4.csv'
reci_pre_new_4000_4.to_csv(output_path, index=True)  # Set index to True here to preserve the line name.

reci_pre_new_4000_4

,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,...,d3991,d3992,d3993,d3994,d3995,d3996,d3997,d3998,d3999,d4000
r1,1433,2868,1611,1954,805,588,3239,1955,796,1612,...,3715,2984,2981,799,582,3722,3246,2429,1425,2156
r2,2052,3117,400,2841,2844,1793,3125,884,2845,402,...,1800,3639,3626,2847,1795,1794,1323,878,61,709
r3,2314,3495,834,1485,1478,3066,3519,412,1520,827,...,1026,3107,3119,1480,3065,1038,2249,427,865,1
r4,1433,2868,1611,1954,805,588,3239,1955,796,1612,...,3715,2984,2981,799,582,3722,3246,2429,1425,2156
r5,1433,2868,1611,1954,805,588,3239,1955,796,1612,...,3715,2984,2981,799,582,3722,3246,2429,1425,2156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
r3996,2991,2315,2048,2497,1238,425,3236,1201,107,1879,...,3400,2276,1939,232,1464,3609,3565,2390,1166,2422
r3997,2931,3642,383,2495,2936,2112,3181,123,1609,319,...,1137,3625,3446,1828,3109,1319,1281,534,115,552
r3998,2931,3642,383,2495,2936,2112,3181,123,1609,319,...,1137,3625,3446,1828,3109,1319,1281,534,115,552
r3999,2258,3411,1039,1561,2263,969,2662,398,320,897,...,2927,3385,3099,574,2554,3303,3219,1416,370,1456
